#### Upper-level functions for building simulations and analysis

In [ ]:
# import needed libraries
import os 
import numpy as np 
import matplotlib.pyplot as plt 
from os.path import basename
from re import sub
import import_ipynb
from SimpleSource_TRAK_tools_2_CHANGE_FE_2 import *
import sys
import subprocess
import statistics

In [ ]:
## define directory where all the sim files will go 
#subdir = 'TRAK_build_analyze_test'
#subdir will need to be included in all of the filenames in the functions! 

#### build-type functions: 

In [ ]:
## run mesh for estat sims 
def run_mesh_estat(fe_x, fe_y, an_x, an_y, meshfile_estat):
    meshfile = '%s'%meshfile_estat
    mesh_file_electrode_spacing2(fe_x, fe_y, an_x, an_y, '%s'%meshfile) # call function to create .min file for estat
    os.system('MESH %s' %meshfile) # run mesh to create .mls & .mou files

In [ ]:
# run estat
def run_estat(estatfile, meshfile_estat, potential, fe_pot): 
    estat_file2('%s'%meshfile_estat, '%s'%estatfile, '%s'%potential, '%s'%fe_pot) # fall function to create .ein file
    os.system('ESTAT %s'%estatfile) # run estat to create .els & .els files

In [ ]:
## run mesh for permag sims 
def run_mesh_permag(meshfile_permag): 
    mesh_permag_file2('%s'%meshfile_permag) # call function to create .min file for permag
    os.system('MESH %s'%meshfile_permag) # run mesh to create .mls & .mou files

In [ ]:
## run permag 
def run_permag(permagfile, meshfile_permag): 
    permag_file2('%s'%meshfile_permag, '%s'%permagfile) # call function to create .pin file
    os.system('PERMAG %s'%permagfile) # run peramg to create .pls & .pou files 

In [ ]:
## run trak 
def run_trak(estatfile, permagfile, trakfile):
    TRAK_file2('%s'%estatfile, '%s'%permagfile, '%s'%trakfile) # call function to create .tin file 
    os.system('TRAK %s'%trakfile) # run trak to create .tls, .tou, .prt, and .eou final files

In [ ]:
# create trak files for looping through demit values 
def demit_loop_files(avg, NCycle, TRAK_demit_array, subdir, fe_x, fe_y, an_x, an_y, trakfile):  #(0.2, 40, np.arange(0.07, 0.25, 0.01), 'TRAK_build_analyze_test'', 5, 2, 50, 5, 'TRAK...' )
    avg_str = str(avg)
    avg_str = avg_str.replace('.', '')
    TRAK_demit_filename_list = []
    for i in TRAK_demit_array: 
        i = round(i, 2)
        demit_str = str(i)
        demit_file = demit_str.replace('.', '')
        TRAK_demit_filename = '%s/%s%s%s%s_avg%sn%s_demt_%s'%(subdir, fe_x, fe_y, an_x, an_y, avg_str, NCycle, demit_file)
        TRAK_demit_filename_list.append(TRAK_demit_filename)
        #run the function to create the TRAK files with the appropraite demit 
        #demit_func('%s'%TRAKfilename, '%s'%TRAK_demit_filename, i)
        avg_demit_ncycle_func('%s'%trakfile, '%s'%TRAK_demit_filename, avg, i, NCycle)

    #print(TRAK_demit_filename_list)
    return(TRAK_demit_filename_list)    

In [ ]:
## can just use run_parallel then to run the demit_loop_file files 
#run_parallel(TRAK_demit_filename_list, parallel_runs, command)

In [ ]:
# create trak files for looping through avg values
def avg_loop_files(demit, NCycle, TRAK_avg_array, subdir, fe_x, fe_y, an_x, an_y, trakfile):
    demit_str = str(demit)
    demit_str = demit_str.replace('.', '')
    #print(demit_str)
    TRAK_avg_filename_list = []
    for i in TRAK_avg_array: 
        i = round(i, 2)
        avg_str = str(i)
        avg_file = avg_str.replace('.', '')
        TRAK_avg_filename = '%s/%s%s%s%s_demt%sn%s_avg_%s'%(subdir, fe_x, fe_y, an_x, an_y, demit_str, NCycle, avg_file)
        TRAK_avg_filename_list.append(TRAK_avg_filename)
        #run the function to create the TRAK files with the appropraite demit 
        #avg_func('%s'%TRAKfilename, '%s'%TRAK_avg_filename, i)
        avg_demit_ncycle_func('%s'%trakfile, '%s'%TRAK_avg_filename, i, demit, NCycle)

    #print(TRAK_avg_filename_list)
    return(TRAK_avg_filename_list)

In [ ]:
## can just use run_parallel then to run the demit_loop_file files 
#run_parallel(TRAK_avg_filename_list, parallel_runs, command)

#### analyze-type functions: 

In [ ]:
# functions to analyze converging demit simulations (plots and average) 
## Test for convergence for demit sims 
#(conv cond: 0.06)
def demit_conv_analysis(conv_condition, TRAK_demit_filename_list, TRAK_demit_array):
    #print(TRAK_demit_filename_list)
    convergence_demit_list = converg_func(TRAK_demit_filename_list, conv_condition)
    demit_list = []
    emitted_current = []
    for i in convergence_demit_list[1]:
        for j in TRAK_demit_array:
            demit_str = str(j.round(2))
            demit_sci =  "{:E}".format(j)
            demit_sci_str = str(demit_sci)
            demit_file = demit_str.replace('.', '')
            demit_str_2 = i[0][-3:]
            if demit_str_2[0] == '_': 
                demit_str_2 = i[0][-2:]
            if demit_file == demit_str_2:
                demit_list.append(j.round(2))
                emitted_current.append(i[2])


    ## plot the current for only the simulations that have a converging current! 
    plt.plot(demit_list, emitted_current, '.')
    plt.xlabel('demit value')
    plt.ylabel('current emitted')
    plt.title('demit current convergence')
    plt.show()

    #legend_conv = []
    for i, j in zip(range(len(convergence_demit_list[1])), emitted_current): 
        current_conv(convergence_demit_list[1][i][0])
        string = convergence_demit_list[1][i][0][-3:]
        if string[0] == '_': 
            string = string[-2:]
        string_val = string[:1] + '.' + string[1:]
        float_val = np.round(float(string_val), 4)
        current_emit = np.round(j, 3)
        #legend_conv.append('demit: %s, I: %s A' %(float_val, current_emit))
    #plt.legend(legend_conv)
    plt.title("current convergence (Demit variance)")
    plt.show()

    #legend_cdens = []
    for i in range(len(convergence_demit_list[1])): 
        cdensity(convergence_demit_list[1][i][0], 775)
        string = convergence_demit_list[1][i][0][-3:]
        if string[0] == '_': 
            string = string[-2:]
        string_val = string[:1] + '.' + string[1:]
        #legend_cdens.append('demit: %s' %string_val)
    #plt.legend(legend_cdens)
    plt.title('current density slice (demit variance)')
    plt.show()

    ## Get the average of the current emitted
    average = np.round(statistics.mean(emitted_current), 4)
    ## Get the stdev of the current emitted
    standarddev = np.round(statistics.stdev(emitted_current), 4)
    print('CURRENT EMITTED: ', average, '\u00B1', standarddev, "A")
    return (average, standarddev)

In [ ]:
# functions to analyze converging avg simulations (plots and average)
## Test for convergence for avg sims 
def avg_conv_analysis(conv_condition, TRAK_avg_filename_list, TRAK_avg_array):
    convergence_avg_list = converg_func(TRAK_avg_filename_list, conv_condition)
    avg_list = []
    emitted_current = []
    for i in convergence_avg_list[1]:
        for j in TRAK_avg_array:
            avg_str = str(j.round(2))
            avg_sci =  "{:E}".format(j)
            avg_sci_str = str(avg_sci)
            avg_file = avg_str.replace('.', '')
            avg_str_2 = i[0][-3:]
            if avg_str_2[0] == '_': 
                avg_str_2 = i[0][-2:]
            if avg_file == avg_str_2:
                avg_list.append(j.round(2))
                emitted_current.append(i[2])

    plt.plot(avg_list, emitted_current, '.')
    plt.title('avg. current convergence')
    plt.xlabel('avg value')
    plt.ylabel('current emitted')
    plt.show()

    #legend_conv = []
    for i, j in zip(range(len(convergence_avg_list[1])), emitted_current): 
        current_conv(convergence_avg_list[1][i][0])
        string = convergence_avg_list[1][i][0][-3:]
        if string[0] == '_': 
            string = string[-2:]
        string_val = string[:1] + '.' + string[1:]
        float_val = np.round(float(string_val), 4)
        current_val = np.round(j, 3)
        #legend_conv.append('avg: %s, I: %s A' %(float_val, current_val ))
    #plt.legend(legend_conv)
    plt.title("current convergence (avg variance)")
    plt.show()

    #legend_cdens = []
    for i in range(len(convergence_avg_list[1])): 
        cdensity(convergence_avg_list[1][i][0], 775)
        string = convergence_avg_list[1][i][0][-3:]
        if string[0] == '_': 
            string = string[-2:]
        string_val = string[:1] + '.' + string[1:]
        #legend_cdens.append('avg: %s' %string_val)
    #plt.legend(legend_cdens)
    plt.title('current density slice (avg variance)')
    plt.show()

    ## Get the average of the current emitted
    average = np.round(statistics.mean(emitted_current), 4)
    ## Get the stdev of the current emitted
    standarddev = np.round(statistics.stdev(emitted_current), 4)
    print('CURRENT EMITTED: ', average, '\u00B1', standarddev, "A")
    return(average, standarddev)

In [ ]:
# functions to analyze converging avg simulations (plots and average)
## Test for convergence for avg sims 
def avg_conv_analysis_cartprod_convg(conv_condition, TRAK_avg_filename_list, TRAK_avg_array):
    convergence_avg_list = converg_func(TRAK_avg_filename_list, conv_condition)
    avg_list = []
    emitted_current = []
    for i in convergence_avg_list[1]:
        for j in TRAK_avg_array:
            avg_str = str(j.round(2))
            avg_sci =  "{:E}".format(j)
            avg_sci_str = str(avg_sci)
            avg_file = avg_str.replace('.', '')
            avg_str_2 = i[0][-3:]
            if avg_str_2[0] == '_': 
                avg_str_2 = i[0][-2:]
            if avg_file == avg_str_2:
                avg_list.append(j.round(2))
                emitted_current.append(i[2])

    for i, j in zip(range(len(convergence_avg_list[1])), emitted_current): 
        current_conv(convergence_avg_list[1][i][0])
        string = convergence_avg_list[1][i][0][-3:]
        if string[0] == '_': 
            string = string[-2:]
        string_val = string[:1] + '.' + string[1:]
        float_val = np.round(float(string_val), 4)
        current_val = np.round(j, 3)
    plt.title("current convergence (avg variance)")

    ## Get the average of the current emitted
    average = np.round(statistics.mean(emitted_current), 4)
    ## Get the stdev of the current emitted
    standarddev = np.round(statistics.stdev(emitted_current), 4)
    print('CURRENT EMITTED: ', average, '\u00B1', standarddev, "A")
    return(average, standarddev)

In [ ]:
# functions to analyze converging avg simulations (plots and average)
## Test for convergence for avg sims 
def avg_conv_analysis_cartprod_dens(conv_condition, TRAK_avg_filename_list, TRAK_avg_array):
    convergence_avg_list = converg_func(TRAK_avg_filename_list, conv_condition)
    avg_list = []
    emitted_current = []
    for i in convergence_avg_list[1]:
        for j in TRAK_avg_array:
            avg_str = str(j.round(2))
            avg_sci =  "{:E}".format(j)
            avg_sci_str = str(avg_sci)
            avg_file = avg_str.replace('.', '')
            avg_str_2 = i[0][-3:]
            if avg_str_2[0] == '_': 
                avg_str_2 = i[0][-2:]
            if avg_file == avg_str_2:
                avg_list.append(j.round(2))
                emitted_current.append(i[2])

    #legend_cdens = []
    for i in range(len(convergence_avg_list[1])): 
        cdensity(convergence_avg_list[1][i][0], 775)
        string = convergence_avg_list[1][i][0][-3:]
        if string[0] == '_': 
            string = string[-2:]
        string_val = string[:1] + '.' + string[1:]
        #legend_cdens.append('avg: %s' %string_val)
    #plt.legend(legend_cdens)
    plt.title('Total current density slice')
    #plt.show()

    ## Get the average of the current emitted
    average = np.round(statistics.mean(emitted_current), 4)
    ## Get the stdev of the current emitted
    standarddev = np.round(statistics.stdev(emitted_current), 4)
    print('CURRENT EMITTED: ', average, '\u00B1', standarddev, "A")
    return(average, standarddev)

In [ ]:
# functions to return the flatness of the total simulations (works for both the demit and avg) 
def flatness(file_list, density_slice, radius): 
    flatness_list = []
    for i in file_list: 
        flatness = dist_flatness(i, density_slice, radius)
        flatness_list.append([i, flatness[0], flatness[1]])
    return(flatness_list)